In [1]:
import collections
collections.Iterable = collections.abc.Iterable
import time
import multiprocessing as mp
import sys
import urx
import numpy as np
import math
import threading
import io
import socket
import struct
from PIL import Image
import cv2
import numpy
import pupil_apriltags as apriltag
import time
from scipy.spatial.transform import Rotation as R
import transforms3d

In [2]:
tag = False
tag_pose = []

In [3]:
def coord_receiver(connection,detector,camera_params,tag_size,robot,goal,home,accuracy):
    global tag,tag_pose,threads_active
    print("coord_reciever thread started")
    
    while threads_active:
        image_len = struct.unpack(
            '<L', connection.read(struct.calcsize('<L')))[0]
        # print('I am here')
        if not image_len:
            break
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        image_stream.seek(0)
        image = Image.open(image_stream)
        opencvImage = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
        result = detector.detect(
            gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=tag_size)

        if result:
            tag = True
            tag_pose = result[0].pose_t
            print('coords=',tag_pose)
        else:
            tag=False
            tag_pose = []
        cv2.imshow('Image', opencvImage)

        current_rob_pose = robot.getl()
        diff = -np.array(current_rob_pose[:3])+np.array(goal[:3])
        v = diff/np.max(np.abs(diff))*0.01
        robot.speedl((v[0],v[1],v[2],0,0,0),0.05,1)
        distance = np.linalg.norm(diff)

        if distance <=accuracy:
            robot.speedl((0,0,0,0,0,0),0.05,1)
            threads_active = False

        if tag and np.linalg.norm(tag_pose) < 0.2:
            print('COLLISION')
            robot.speedl((0,0,0,0,0,0),0.05,1)
            robot.movel((home[0],home[1],home[2],home[3],home[4],home[5]),0.01,0.01)
            threads_active = False
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            threads_active = False
        time.sleep(0.01)

        


In [ ]:
def manip_control(robot,goal,home,accuracy, conn):
    global threads_active
    print("manip_control process started")
    while threads_active:
        current_rob_pose = robot.getl()
        diff = -np.array(current_rob_pose[:3])+np.array(goal[:3])
        v = diff/np.max(np.abs(diff))*0.01
        robot.speedl((v[0],v[1],v[2],0,0,0),0.05,1)
        distance = np.linalg.norm(diff)

        if distance <=accuracy:
            robot.speedl((0,0,0,0,0,0),0.05,1)
            threads_active = False

        if conn.recv == 1:
            print('Recieved collision')
            robot.speedl((0,0,0,0,0,0),0.05,1)
            robot.movel((home[0],home[1],home[2],home[3],home[4],home[5]),0.01,0.01)
            threads_active = False
            
        time.sleep(0.01)

        

In [ ]:
def camera_control(connection,detector,camera_params,tag_size,conn):
    global tag,tag_pose,threads_active
    print("camera control process started")
    
    while threads_active:
        image_len = struct.unpack(
            '<L', connection.read(struct.calcsize('<L')))[0]
        # print('I am here')
        if not image_len:
            break
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        image_stream.seek(0)
        image = Image.open(image_stream)
        opencvImage = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
        result = detector.detect(
            gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=tag_size)

        if result:
            tag = True
            tag_pose = result[0].pose_t
            print('coords=',tag_pose)
        else:
            tag=False
            tag_pose = []
        cv2.imshow('Image', opencvImage)

        if tag and np.linalg.norm(tag_pose) < 0.2:
            conn.send(1)
            print('COLLISION Sent')
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            threads_active = False
        time.sleep(0.01)

In [4]:
camera_params = (506.19083684, 508.36108854,
                 317.93111342, 243.12403806)
tag_size = 0.0375

UDP_IP = "127.0.0.1"
UDP_PORT = 5065
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket = socket.socket()
server_socket.bind(('0.0.0.0', 8000))
server_socket.listen(0)
connection = server_socket.accept()[0].makefile('rb')

accuracy = 0.001

detector = apriltag.Detector(
    families='tag36h11',
    nthreads=10,
    quad_decimate=0.5,
    quad_sigma=0,
    refine_edges=1,
    decode_sharpening=0.6,
    debug=0)



In [5]:
rob = urx.Robot('192.168.88.60')
home = [-0.297,
 -0.142,
 0.49,
 0.11,
 -1.52,
 0.19]
goal = [-0.41,
 -0.142,
 0.49,
 0.11,
 -1.52,
 0.19]

In [6]:
rob.getl()

[-0.4130098640207345,
 -0.1422330594335932,
 0.49061181924385733,
 0.10995041598225339,
 -1.5199074785624982,
 0.18999002588060435]

In [8]:
threads_active=True
conn1, conn2 = mp.Pipe()
camera_control_process = mp.Process(
target=camera_control,
daemon=True,
args=(connection,detector,camera_params,tag_size,conn2)
)
camera_control_process.start()

manip_control_process = mp.Process(
    target=manip_control,
    daemon=True,
    args=(rob,goal,home,accuracy,conn1)
)
manip_control_process.start()

#let's see if it works
manip_control_process.join()
camera_control_process.join()

coord_reciever thread started


coords= [[-0.00237117]
 [-0.01241012]
 [ 0.1918343 ]]
COLLISION


In [8]:
threads_active=False
connection.close()

In [7]:
rob.movel(goal,0.05,0.01)

<Transform:
<Orientation: 
array([[ 0.04013191, -0.19197181, -0.98057955],
       [ 0.05520934,  0.98029698, -0.18965695],
       [ 0.99766796, -0.04652586,  0.04993982]])>
<Vector: (-0.40998, -0.14199, 0.49002)>
>